In [588]:
import xlwings as xw
import pandas as pd

In [589]:
recl_path = r"C:\Users\nil.monreal\Desktop\Nil\EZ Notes\Python Scripts\Pyhton Scripts\AR Automation\AR Reports\2.- RECL OCTOBER 2024.xlsx"
ar51_path = r"C:\Users\nil.monreal\Desktop\Nil\EZ Notes\Python Scripts\Pyhton Scripts\AR Automation\Closing Reports Ry4\AR 5 1 DISTRIBUTION REPORT SEPTEMBER 2024.xlsx"

In [590]:
# Step 2: Open the first file (recl_path) and the AR 5 1 sheet
recl_wb = xw.Book(recl_path, update_links=False)  # Open the Recl workbook
recl_sheet = recl_wb.sheets['AR 5 1']  # Access the 'AR 5 1' sheet

In [591]:
# Step 3: Remove any existing filters (reset AutoFilters)
if recl_sheet.api.AutoFilterMode:
    recl_sheet.api.AutoFilterMode = False

In [592]:
# Step 4: Delete all content from the sheet (clear all data)
recl_sheet.clear()  # This clears the entire sheet's content

In [593]:
# Step 5: Open the second file (ar51_path) and copy content from its AR 5 1 sheet
ar51_wb = xw.Book(ar51_path, update_links=False)  # Open the AR 5 1 workbook
ar51_sheet = ar51_wb.sheets['AR 5 1']  # Access the 'AR 5 1' sheet

In [594]:
# Copy all data from AR 5 1 sheet in ar51 file
ar51_data = ar51_sheet.used_range.value  # Get the entire data from the second file

In [595]:
# Paste the data into the AR 5 1 sheet of the recl file
recl_sheet.range('A1').value = ar51_data  # Paste the data starting at A1

In [596]:
ar51_wb.close()

In [597]:
last_row = recl_sheet.range('M1').end('down').row

recl_sheet.range(f"A5:L{last_row}").api.AutoFilter(Field=12, Criteria1=['CDZ', 'OCT', 'SAF'], Operator=7)


12.0

In [598]:
filtered_data = recl_sheet.range(f"A6:L{last_row}").options(pd.DataFrame, header=1, index=False).value


# Create a DataFrame from the filtered rows
df = pd.DataFrame(filtered_data)

# Convert NaN values to zero
df.fillna(0, inplace=True)

df

,GL-code,Description,Entity,Tran-date,Document,Debit-amt,Credit-amt,Customer,Name,Tr,Jrnl,Group
0,---------,---------------,---------,---------,------------,---------------,-----------------,---------,-----------------------------,---,------,-----
1,1092.0,EZ CITI 8028,EZ,2024-09-04 00:00:00,OFFSETSEP,35023.81,35023.81,EZ1094,YABORA INDUSTRIA AERONAUTICA,PT,1455.0,EMB
2,0,0,0,2024-09-05 00:00:00,3417713427.0,285419.04,0,EZ1001,EMBRAER SA,PT,1455.0,EMB
3,0,0,0,2024-09-05 00:00:00,3417713427.0,179391.87,0,EZ1023,EMBRAER SA,PT,1455.0,EMB
4,0,0,0,2024-09-05 00:00:00,3417713427.0,903.74,0,EZ1031,EMBRAER SA,PT,1455.0,EMB
...,...,...,...,...,...,...,...,...,...,...,...,...
1048565,0,0,0,0,0,0,0,0,0,0,0,0
1048566,0,0,0,0,0,0,0,0,0,0,0,0
1048567,0,0,0,0,0,0,0,0,0,0,0,0
1048568,0,0,0,0,0,0,0,0,0,0,0,0


In [599]:
# Filter rows where 'Group' is in the given list
df = df[df['Group'].isin(['CDZ', 'OCT', 'SAF','cdz'])]

# Convert 'Document' column to strings using .loc to ensure it's set correctly
df.loc[:, 'Document'] = df['Document'].astype(str)

# Remove the '.0' from the end of the values in 'Document' column
df.loc[:, 'Document'] = df['Document'].str.replace('.0', '', regex=False)

df = df.drop(['GL-code', 'Description', 'Entity'], axis=1)

df = df[df['Debit-amt'] != 0]

# Filter rows where 'Document' starts with '341'
df3 = df[df['Document'].str.startswith('3')]

In [600]:
df3  = df3[~df3[['Debit-amt', 'Credit-amt']].apply(frozenset, axis=1).duplicated(keep='first')]

In [601]:
df3

,Tran-date,Document,Debit-amt,Credit-amt,Customer,Name,Tr,Jrnl,Group
6,2024-09-06 00:00:00,3417746720,41271.22,0,EZ1005,SAFRAN CABIN INC,PT,1455.0,SAF
7,2024-09-06 00:00:00,3417746720,6061.57,0,EZ1017,SAFRAN CABIN INC,PT,1455.0,SAF
8,2024-09-05 00:00:00,3417717667,74.88,0,EZ1006,SAFRAN CABIN BRAZIL LTDA,PT,1455.0,CDZ
10,2024-09-06 00:00:00,3417746603,1176.9,0,EZ1098,SAFRAN CABIN INC,PT,1455.0,cdz
11,2024-09-06 00:00:00,3417746720,3066.0,0,EZ1017,SAFRAN CABIN INC,PT,1457.0,SAF
14,2024-09-12 00:00:00,3418112503,93898.0,0,EZ1098,SAFRAN CABIN INC,PT,1457.0,cdz
34,2024-09-18 00:00:00,3418534270,144000.43,0,EZ1005,SAFRAN CABIN INC,PT,1459.0,SAF
35,2024-09-18 00:00:00,3418534270,11349.88,0,EZ1017,SAFRAN CABIN INC,PT,1459.0,SAF
36,2024-09-18 00:00:00,3418585458,1289.15,0,EZ1120,SAFRAN ELECTRICAL COMPONENTS,PT,1459.0,CDZ
37,2024-09-20 00:00:00,3418724109,1576.32,0,EZ1098,SAFRAN CABIN INC,PT,1459.0,cdz


In [602]:
row_count = df3.shape[0]
row_count

12

In [603]:
recl_sheet2 = recl_wb.sheets["RECL VENTAS-PAGOS"]
last_row_rcl = recl_sheet2.range('B1').end('down').row

if row_count > last_row_rcl:
    # Start inserting from row 13 to preserve data above
    if last_row_rcl >= 12:
        recl_sheet2.range(f'{row_count}:{last_row_rcl+1}').api.Insert()

elif row_count < last_row_rcl:
    # Start inserting from row 13 to preserve data above
    if row_count >= 12:
        recl_sheet2.range(f'{row_count}:{last_row_rcl+1}').api.Delete()

else:
    pass




In [604]:
recl_sheet2.range(f'B2:J{last_row_rcl}').clear()

In [605]:
df3[['Debit-amt', 'Credit-amt']] = df3[['Credit-amt', 'Debit-amt']]
df3

,Tran-date,Document,Debit-amt,Credit-amt,Customer,Name,Tr,Jrnl,Group
6,2024-09-06 00:00:00,3417746720,0,41271.22,EZ1005,SAFRAN CABIN INC,PT,1455.0,SAF
7,2024-09-06 00:00:00,3417746720,0,6061.57,EZ1017,SAFRAN CABIN INC,PT,1455.0,SAF
8,2024-09-05 00:00:00,3417717667,0,74.88,EZ1006,SAFRAN CABIN BRAZIL LTDA,PT,1455.0,CDZ
10,2024-09-06 00:00:00,3417746603,0,1176.9,EZ1098,SAFRAN CABIN INC,PT,1455.0,cdz
11,2024-09-06 00:00:00,3417746720,0,3066.0,EZ1017,SAFRAN CABIN INC,PT,1457.0,SAF
14,2024-09-12 00:00:00,3418112503,0,93898.0,EZ1098,SAFRAN CABIN INC,PT,1457.0,cdz
34,2024-09-18 00:00:00,3418534270,0,144000.43,EZ1005,SAFRAN CABIN INC,PT,1459.0,SAF
35,2024-09-18 00:00:00,3418534270,0,11349.88,EZ1017,SAFRAN CABIN INC,PT,1459.0,SAF
36,2024-09-18 00:00:00,3418585458,0,1289.15,EZ1120,SAFRAN ELECTRICAL COMPONENTS,PT,1459.0,CDZ
37,2024-09-20 00:00:00,3418724109,0,1576.32,EZ1098,SAFRAN CABIN INC,PT,1459.0,cdz


In [606]:
last_row_rcl = recl_sheet2.range('B1').end('down').row

last_row_rcl

16

In [607]:
#Copying values and setting the sum()
recl_sheet2.range('B2').value = df3.values

last_row_rcl = recl_sheet2.range('B1').end('down').row

# recl_sheet2.range(f'{last_row_rcl+1}:{last_row_rcl+1}').api.Insert()

recl_sheet2.range(f'D{last_row_rcl+1}').formula = f"=sum(D{row_count+1}:D2)"
recl_sheet2.range(f'E{last_row_rcl+1}').formula = f"=sum(E{row_count+1}:E2)"
recl_sheet2.range(f'F{last_row_rcl+1}').formula = f"=(E{last_row_rcl+1}-D{last_row_rcl+1})"



In [608]:
# Start searching from the given last_non_empty_row
start_row = recl_sheet2.range(f'A1').end('down').row  # You want to start searching after the last non-empty row

last_row_rcl2 = recl_sheet2.range(f'B{start_row}').end('down').row

start_row

#If the cell is empty, clear the entire row
# recl_sheet2.range(f'{start_row}:{last_row_rcl2}').clear()

16

In [609]:
# Filter rows where 'Document' starts with 'CH and CM'
dfrest_ch = df[df['Document'].str.startswith('CH')]
dfrest_cm = df[df['Document'].str.startswith('CM')]

dfrest_cm[['Debit-amt', 'Credit-amt']] = dfrest_cm[['Credit-amt', 'Debit-amt']]

dfrest = pd.concat([dfrest_ch, dfrest_cm])

C:\Users\nil.monreal\AppData\Local\Temp\ipykernel_13048\3445235966.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfrest_cm[['Debit-amt', 'Credit-amt']] = dfrest_cm[['Credit-amt', 'Debit-amt']]


In [610]:
row_count2 = dfrest.shape[0] + last_row_rcl + 3

In [611]:
row_count2

229

In [612]:
recl_sheet2.range(f'B{start_row+1}:J{last_row_rcl2}').clear()

In [613]:
recl_sheet2.range(f'B{start_row+1}').value = dfrest.values

In [614]:
last_row_rcl2 = recl_sheet2.range(f'B{start_row}').end('down').row
last_row_rcl2

229

In [615]:
recl_sheet2.range(f'D{last_row_rcl2+5}').formula = f"=sum(D{last_row_rcl2}:D{start_row+1})"
recl_sheet2.range(f'E{last_row_rcl2+5}').formula = f"=sum(E{last_row_rcl2}:E{start_row+1})"
recl_sheet2.range(f'F{last_row_rcl2+5}').formula = f"=(D{last_row_rcl2+5}-E{last_row_rcl2+5})"

In [616]:
# recl_wb.save()